In [4]:
import os
import csv
from datetime import datetime, timezone
import influxdb_client
from influxdb_client.client.write_api import ASYNCHRONOUS
from influxdb_client.client.exceptions import InfluxDBError
from influxdb_client import Point
from urllib3.exceptions import NewConnectionError

INFLUX_URL = "http://influxdb2:8086"
INFLUX_TOKEN = "MyInitialAdminToken0=payosno"

client = None
write_api = None

try:
    client = influxdb_client.InfluxDBClient(
        url=INFLUX_URL,
        token=INFLUX_TOKEN,
        org="docs"
    )

    write_api = client.write_api(
        batch=5000,
        write_api=ASYNCHRONOUS,
    )
    
except (InfluxDBError, NewConnectionError) as e:
    print("[ERROR] Error al conectar con InfluxDB:")
    print(f"   Detalle: {e}")



ficheros = os.listdir("./UT02/data")

contadorPuntos = 0

for fic in ficheros:
    with open("./UT02/data/"+fic, newline="", encoding="utf-8") as f:
        reader = csv.reader(f)

        next(reader, None)

        for row in reader:
            name = row[1]
            symbol = row[2]

            date_ = row[3]
            dt = datetime.strptime(date_,"%Y-%m-%d %H:%M:%S")
            dt_utc = dt.replace(tzinfo=timezone.utc)
            time_date = dt_utc.isoformat().replace('+00:00','Z')

            high = float(row[4])
            low = float(row[5])
            close_ = float(row[6])
            open_ = float(row[7])
            volume = float(row[8])
            marketcap = float(row[9])

            p = Point("daily_quote") \
                .tag("Symbol", symbol) \
                .tag("Name", name) \
                .time(time_date) \
                .field("Close", close_) \
                .field("High", high) \
                .field("Low", low) \
                .field("Open", open_) \
                .field("Volume", volume) \
                .field("Marketcap", marketcap)
            
            contadorPuntos = contadorPuntos + 1
            
            write_api.write(bucket="crypto_raw", org="docs", record=p)
    print(f"Fichero {fic} procesado")
write_api.close()

print("------------------------------------")
print(f"Puntos insertados: {contadorPuntos}")




Fichero coin_Aave.csv procesado
Fichero coin_BinanceCoin.csv procesado
Fichero coin_Bitcoin.csv procesado
Fichero coin_Cardano.csv procesado
Fichero coin_ChainLink.csv procesado
Fichero coin_Cosmos.csv procesado
Fichero coin_CryptocomCoin.csv procesado
Fichero coin_Dogecoin.csv procesado
Fichero coin_EOS.csv procesado
Fichero coin_Ethereum.csv procesado
Fichero coin_Iota.csv procesado
Fichero coin_Litecoin.csv procesado
Fichero coin_Monero.csv procesado
Fichero coin_NEM.csv procesado
Fichero coin_Polkadot.csv procesado
Fichero coin_Solana.csv procesado
Fichero coin_Stellar.csv procesado
Fichero coin_Tether.csv procesado
Fichero coin_Tron.csv procesado
Fichero coin_Uniswap.csv procesado
Fichero coin_USDCoin.csv procesado
Fichero coin_WrappedBitcoin.csv procesado
Fichero coin_XRP.csv procesado
------------------------------------
Puntos insertados: 37082
